In [2]:
##IMPORTS
from __future__ import division
import rhinoinside
rhinoinside.load()

import clr
clr.AddReference("Grasshopper")
from Grasshopper import DataTree
from Grasshopper.Kernel.Data import GH_Path as Path

import System
import Rhino.Geometry as rg
from Rhino.Geometry import Point3d, Plane, Brep, Vector3d
from Rhino.Geometry import Surface as RhinoSurface

from Rhino.Geometry import Transform
from Rhino.FileIO import File3dm

import ladybug.epw as epw

from ladybug_geometry.geometry2d.pointvector import Vector2D

from lbt_recipes.version import check_radiance_date

#IMPORTS FOR INCIDENT RADIATION
from ladybug.viewsphere import view_sphere
from ladybug.graphic import GraphicContainer
from ladybug.legend import LegendParameters
from ladybug.color import Colorset

from ladybug_rhino.config import conversion_to_meters
from ladybug_rhino.togeometry import to_joined_gridded_mesh3d, to_vector2d
from ladybug_rhino.fromgeometry import from_mesh3d, from_point3d, from_vector3d
from ladybug_rhino.intersect import intersect_mesh_rays, join_geometry_to_mesh

import ladybug.analysisperiod as ap

from ladybug_radiance.skymatrix import SkyMatrix
import ladybug.analysisperiod as ap

from datetime import datetime, timedelta
import collections
import array
import math
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



# check the installed Radiance date and get the path to the gemdaymtx executable
check_radiance_date()

In [3]:
#FONCTIONS GH       
def objectify_output(object_name, output_data):
    """Wrap data into a single custom Python object that can later be de-serialized.

    This is meant to address the same issue as the wrap_output method but it does
    so by simply hiding the individual items from the Grasshopper UI within a custom
    parent object that other components can accept as input and de-objectify to
    get access to the data. This strategy is also useful for the case of standard
    object types like integers where the large number of data points slows down
    the Grasshopper UI when they are output.

    Args:
        object_name: Text for the name of the custom object that will wrap the data.
            This is how the object will display in the Grasshopper UI.
        output_data: A list of data to be stored under the data property of
            the output object.
    """
    class Objectifier(object):
        """Generic class for objectifying data."""

        def __init__(self, name, data):
            self.name = name
            self.data = data

        def ToString(self):
            return '{} ({} items)'.format(self.name, len(self.data))

    return Objectifier(object_name, output_data)

def de_objectify_output(objectified_data):
    """Extract the data from an object that was output from the objectify_output method.

    Args:
        objectified_data: An object that has been output from the objectify_output
            method for which data will be returned.
    """
    return objectified_data.data

def text_objects(text, plane, height, font='Arial',
                 horizontal_alignment=0, vertical_alignment=5):
    """Generate a Bake-able Grasshopper text object from a text string and ladybug Plane.

    Args:
        text: A text string to be converted to a a Grasshopper text object.
        plane: A Ladybug Plane object to locate and orient the text in the Rhino scene.
        height: A number for the height of the text in the Rhino scene.
        font: An optional text string for the font in which to draw the text.
        horizontal_alignment: An optional integer to specify the horizontal alignment
             of the text. Choose from: (0 = Left, 1 = Center, 2 = Right)
        vertical_alignment: An optional integer to specify the vertical alignment
             of the text. Choose from: (0 = Top, 1 = MiddleOfTop, 2 = BottomOfTop,
             3 = Middle, 4 = MiddleOfBottom, 5 = Bottom, 6 = BottomOfBoundingBox)
    """
    txt = Rhino.Display.Text3d(text, from_plane(plane), height)
    txt.FontFace = font
    txt.HorizontalAlignment = AlignmentTypes.horizontal(horizontal_alignment)
    txt.VerticalAlignment = AlignmentTypes.vertical(vertical_alignment)
    return TextGoo(txt)

In [4]:
#FONCTIONS GEOMETRIE
def create_surface(x, y, width, length):
    """
    Create a simple NURBS Rhino Surface, from one of its four angles and its dimensions (width, length)

    Args :
    x (float) : X-coordinate of origin angle
    y (float) : Y-coordinate of origin angle
    width (float) : width of the surface
    length (float) : length of the surface

    Return :
    surface (Rhino.Geometry.NurbsSurface) : Surface created
    """
    pt1 = rg.Point3d(x, y, 0)
    pt2 = rg.Point3d(x, y + width, 0)
    pt3 = rg.Point3d(x + length, y + width, 0)
    pt4 = rg.Point3d(x + length, y, 0)
    
    # Create a point list
    points = [pt1, pt2, pt3, pt4]

    # Create a surface from the previous point list, and return it
    surface = rg.NurbsSurface.CreateFromCorners(pt1, pt2, pt3, pt4)

    return surface
    
def create_panel_grid(grid_size, nb_rangs, nb_pvp_rangs, width, length, height, column_spacing, row_spacing, angle_orientation, angle_variable, semi_transp = False, bande = None, void_space = None):
    """
    According to the parameters, creates a grid of surfaces, later converted into Breps, representing the solar panels.

    Args :
    grid_size (float) : total size of the panel grid ofr simulation (can and will differ from the actual solar panel field)
    nb_rangs (int) : Number of PVP per rows
    nb_pvp_rangs (int) : Number of rows
    width (float) : Width of a PVP
    length (float) : Length of a PVP
    column_spacing (float) : Distance between two PVP rows (forced value (= ENTRAXE))
    row_spacing (float) : Distance between two PVP inside a row (can be equals to 0, forced value (=grid_size/nb_rangs))
    angle_orientation (int) : Difference between the field axis (PVP supports' axis) and the North-South axis
    angle_variable (int) : tilt of the PVPs
    semi_transp (bool) : False = classic PV Panel, one surface
                         True = Semi-transparent PV panel, composed by 4 strips of PV cells and 3 strips of space inbetween the cells.
    bande (float) : width of the PV cells strip, in case of semi transp panel
    void_space (float) : width of the void between PV cells strip, in case of semi transp panel 
    
    Return :
    final_brep (Rhino.Geometry.Brep) : Brep containing all the PVPs

    """
    final_brep = rg.Brep()

    for i in range(nb_rangs):
        for j in range(nb_pvp_rangs):
            # (x;y) are the coordinate of the current panel created
            x = (j * column_spacing) - grid_size / 2
            y = (i * row_spacing) - grid_size / 2
            surface = []

            if semi_transp == True:
                for h in range(0,4) :
                    strip_x = x + h * (bande + void_space)
                    surface.append(create_surface(strip_x, y, length, bande))
            else:    
                surface.append(create_surface(x, y, length, width))
                
            # Rotation around the X axis for each Brep individually
            center_of_surface = Point3d(x + width / 2, y + length / 2, 0)
            rotation_x = Transform.Rotation(angle_variable * (System.Math.PI / 180), Vector3d.YAxis, center_of_surface)
            for surf in surface:
                surf.Transform(rotation_x)
                # Adding it to the final returned brep
                final_brep.Append(surf.ToBrep())       
 
    # Global rotation around the Z axis at the center of the grid (origin)
    rotation_z = Transform.Rotation(-angle_orientation * (System.Math.PI / 180), Vector3d.ZAxis, rg.Point3d.Origin)
    final_brep.Transform(rotation_z)

    # Global translation on the Z axis, equivalent to the panel rotation axis height
    translation_z = Transform.Translation(0,0,height)
    final_brep.Transform(translation_z)

    # Inversion of the orientation of the normals for proper operation of the Incident Radiation function
    final_brep.Flip()

    return final_brep

def create_sensor_grid_2(land_orientation, x, y, culture = False, entraxe = None, rampant = None, nb_lignes = None, grid_size = None):
    """
    Creates the surface from which the Incident Radiation will measure the ground irradiance.

    Args :
    land_orientation (int) : Difference between the field axis (PVP supports' axis) and the North-South axis.
    culture (bool): If True, the measure surface will correspond to a rectangle that will fit under an arboriculture solar panel (used in vine for now)
                    If False, it will correspond to a square, that we could use in field crops.
    x (float) : X-coordinate of the area of measures.
    y (float) : Y-coordinate of the area of measures.
    entraxe (float) : distance between 2 rows of panels supports. None if culture is not arboriculture.
    rampant (float) : width of a pvp panel. None if culture is not arboriculture.
    nb_lignes (int) : number of pvp panels rows (= int(GRID_SIZE / ENTRAXE), calculated in settings)
    grid_size (float) : 
    
    Return:
    sensor_brep_list (list(Rhino.Geometry.Brep)) : Surface created from create_surface function, then converted to a brep, then rotated and translated
                                                    if in the case of arboriculture.

    """
    brep_gen = rg.Brep()
    sensor_brep_list = []
    _TAILLE_SENSOR = 5 #Sensor side length = 5 mètres

    if culture == True:
        # Creating the coordinates of the first point of a measure grid that fits under an arboriculture PVP 
        # (irradiance is not interesting in between vine rows)
        if type(x) == type(None) :
            x_arbo = math.floor(nb_lignes/2)*entraxe + rampant/2 - 0.5 - grid_size/2
            y_arbo = -2.5
        else : x_arbo, y_arbo = x, y

        # Create a surface at the calculated more or less center of the grid UNDER a panel, length = 5 and width = 1 m
        s = create_surface(x_arbo,y_arbo,_TAILLE_SENSOR,1) 
        brep_gen.Append(s.ToBrep())
        
        # Rotating the measure area to align it with the panels, using the same method and geometry references (ZAxis and Origin)
        rotation_z = rg.Transform.Rotation(-land_orientation * (System.Math.PI / 180), Vector3d.ZAxis, rg.Point3d.Origin)
        brep_gen.Transform(rotation_z)

    if culture == False:
        # Create a square with TAILLE_SENSOR is the size of a side, center of the square is the origin of the Rhino Environment
        surface_brep = create_surface(-_TAILLE_SENSOR / 2, -_TAILLE_SENSOR / 2 , _TAILLE_SENSOR, _TAILLE_SENSOR)
        brep_gen.Append(surface_brep.ToBrep())
    
    brep_gen.Flip()
    sensor_brep_list.append(brep_gen)

    return sensor_brep_list

def export_brep_to_file(brep, filename, new_file = True):
    """
    Creates a 3dm file to visualize the objects, mainly Breps, created wiht python scripts.

    Args:
    brep (Rhino.Geometry.Brep): Brep to visualize
    filename (str): Name of the output file.

    Return:
    Nothing, the file is created

    """
    if new_file == False:
        file = File3dm.Read(filename)
        if brep:
            file.Objects.AddBrep(brep)
            file.Write(filename,7)  # Rewrite the file with the new geometry
            print("La nouvelle géométrie a été ajoutée au fichier existant.")
        else:
            print("Erreur lors de la création de la surface.")

    else :
        file = File3dm()
        file.Settings.ModelUnitSystem = Rhino.UnitSystem.Meters
        file.Objects.AddBrep(brep)
        file.Write(filename, 7)  # 7 corresponding to Rhino 7

In [5]:
#FONCTIONS RADIANCE
def create_sky_matrix(_location, _direct_rad, _diffuse_rad, _folder_ = None, north_ = None, _hoys_ = None, h_dens = True):
    """
    Get a matrix containing radiation values from each patch of a sky dome.
    _
    Creating this matrix is a necessary pre-step before doing incident radiation
    analysis with Rhino geometry or generating a radiation rose.
    _
    This component uses Radiance's gendaymtx function to calculate the radiation
    for each patch of the sky. Gendaymtx is written by Ian Ashdown and Greg Ward.
    Morere information can be found in Radiance manual at:
    http://www.radiance-online.org/learning/documentation/manual-pages/pdfs/gendaymtx.pdf
    -

    Args:
        north_: A number between -360 and 360 for the counterclockwise
            difference between the North and the positive Y-axis in degrees.
            90 is West and 270 is East. This can also be Vector for the
            direction to North. (Default: 0)
        _location: A ladybug Location that has been output from the "LB Import EPW"
            component or the "LB Construct Location" component.
        _direct_rad: An annual hourly DataCollection of Direct Normal Radiation such
            as that which is output from the "LB Import EPW" component or the
            "LB Import STAT" component.
        _diffuse_rad: An annual hourly DataCollection of Diffuse Horizontal Radiation
            such as that which is output from the "LB Import EPW" component or
            the "LB Import STAT" component.
        _hoys_: A number or list of numbers between 0 and 8760 that respresent
            the hour(s) of the year for which to generate the sky matrix. The
            "LB Calculate HOY" component can output this number given a month,
            day and hour. The "LB Analysis Period" component can output a
            list of HOYs within a certain hour or date range. By default,
            the matrix will be for the entire year.
        high_density_: A Boolean to indicate whether the higher-density Reinhart sky
            matrix should be generated (True), which has roughly 4 times the sky
            patches as the (default) original Tregenza sky (False). Note that,
            while the Reinhart sky has a higher resolution and is more accurate,
            it will result in considerably longer calculation time for incident
            radiation studies. The difference in sky resolution can be observed
            with the "LB Sky Dome" component. (Default: False).
        _ground_ref_: A number between 0 and 1 to note the average ground reflectance
            that is associated with the sky matrix. (Default: 0.2).
        _folder_: The folder in which the Radiance commands are executed to
            produce the sky matrix. If None, it will be written to Ladybug's
            default EPW folder.

    Returns:
        report: ...
        sky_mtx: A sky matrix object containing the radiation coming from each patch
            of the sky. This can be used for a radiation study, a radition rose,
            or a sky dome visualization. It can also be deconstructed into its
            individual values with the "LB Deconstruct Matrix" component.
    """
    if north_ is not None:  # process the north_
        try:
            north_ = math.degrees(
                to_vector2d(north_).angle_clockwise(Vector2D(0, 1)))
        except AttributeError:  # north angle instead of vector
            north_ = float(north_)
    if _hoys_ is not None: # process if hoys are specified or not
        sky_mtx = SkyMatrix.from_components(
            _location, _direct_rad, _diffuse_rad, 
            _hoys_, high_density = h_dens) #, north_, ground_r
        if _folder_:
            sky_mtx.folder = _folder_
    else:
        # create the sky matrix object
        sky_mtx = SkyMatrix.from_components(
            _location, _direct_rad, _diffuse_rad, high_density = h_dens) #_hoys_, north_, high_density_, ground_r  
        if _folder_:
            sky_mtx.folder = _folder_

    return sky_mtx

def incident_radiation(_sky_mtx_in_IR, _geometry, context_, _offset_dist_, _finesse = 1, irradiance_ = True, legend_par_ = None):
    """
    Calculate the incident radiation on geometry using a sky matrix from the "Cumulative
    Sky Matrix" component.
    _
    Such studies of incident radiation can be used to apprxomiate the energy that can
    be collected from photovoltaic or solar thermal systems. They are also useful
    for evaluating the impact of a building's orientation on both energy use and the
    size/cost of cooling systems. For studies of photovoltaic potential or building
    energy use impact, a sky matrix from EPW radiation should be used. For studies
    of cooling system size/cost, a sky matrix derived from the STAT file's clear sky
    radiation should be used.
    _
    NOTE THAT NO REFLECTIONS OF SOLAR ENERGY ARE INCLUDED
    IN THE ANALYSIS PERFORMED BY THIS COMPONENT.
    _
    Ground reflected irradiance is crudely acounted for by means of an emissive
    "ground hemisphere," which is like the sky dome hemisphere and is derived from
    the ground reflectance that is associated with the connected _sky_mtx. This
    means that including geometry that represents the ground surface will effectively
    block such crude ground reflection.
    _
    Also note that this component uses the CAD environment's ray intersection methods,
    which can be fast for geometries with low complexity but does not scale well for
    complex geometries or many test points. For such complex cases and situations
    where relfection of solar energy are important, honeybee-radiance should be used.
    -

    Args:
        _sky_mtx_in_IR: A Sky Matrix from the "LB Cumulative Sky Matrix" component or the
            "LB Benefit Sky Matrix" component, which describes the radiation
            coming from the various patches of the sky. The "LB Sky Dome"
            component can be used to visualize any sky matrix to understand
            its relationship to the test geometry.
        _geometry: Rhino Breps and/or Rhino Meshes for which incident radiation analysis
            will be conducted. If Breps are input, they will be subdivided using
            the _grid_size to yeild individual points at which analysis will
            occur. If a Mesh is input, radiation analysis analysis will be
            performed for each face of this mesh instead of subdividing it.
        context_: Rhino Breps and/or Rhino Meshes representing context geometry
            that can block solar radiation to the test _geometry.
        _offset_dist_: A number for the distance to move points from the surfaces
            of the input _geometry.  Typically, this should be a small positive
            number to ensure points are not blocked by the mesh. (Default: 10 cm
            in the equivalent Rhino Model units).
        _finesse: A positive number in Rhino model units for the size of grid
            cells at which the input _geometry will be subdivided for incident
            radiation analysis. The smaller the grid size, the higher the
            resolution of the analysis and the longer the calculation will take.
            So it is recommended that one start with a large value here and
            decrease the value as needed. However, the grid size should usually
            be smaller than the dimensions of the smallest piece of the _geometry
            and context_ in order to yield meaningful results.
        irradiance_: Boolean to note whether the study should output units of cumulative
            Radiation (kWh/m2) [False] or units of average Irradiance (W/m2)
            [True].  (Default: False).
        legend_par_: Optional legend parameters from the "LB Legend Parameters"
            that will be used to customize the display of the results.
        _cpu_count_: An integer to set the number of CPUs used in the execution of the
            intersection calculation. If unspecified, it will automatically default
            to one less than the number of CPUs currently available on the
            machine or 1 if only one processor is available.
        _run: Set to "True" to run the component and perform incident radiation
            analysis.

    Returns:
        report: ...
        points: The grid of points on the test _geometry that are be used to perform
            the incident radiation analysis.
        results: A list of numbers that aligns with the points. Each number indicates
            the cumulative incident radiation received by each of the points
            from the sky matrix in kWh/m2.
        total: A number for the total incident solar energy falling on all input geometry
            in kWh. Note that, unlike the radiation results above, which are
            normlaized by area, these values are not area-normalized and so
            the input geometry must be represented correctly in the Rhino
            model's unit system in order for this output to be meaningful.
        mesh: A colored mesh of the test _geometry representing the cumulative
            incident radiation received by the input _geometry.
        legend: A legend showing the kWh/m2 that correspond to the colors of the mesh.
        title: A text object for the study title.
        int_mtx_to_RTIR: A Matrix object that can be connected to the "LB Deconstruct Matrix"
            component to obtain detailed patch-by-patch results of the study.
            Each sub-list of the matrix (aka. branch of the Data Tree) represents
            one of the points used for analysis. The length of each sub-list
            matches the number of sky patches in the input sky matrix (145 for
            the default Tregenza sky and 577 for the high_density Reinhart sky).
            Each value in the sub-list is a value between 0 and 1 indicating the
            relationship between the point and the patch of the sky. A value of
            "0", indicates that the patch is not visible for that point at all
            while a value of "1" indicates that the patch hits the surface that
            the point represents head on.
    """

    # set the default offset distance and _cpu_count
    _offset_dist_ = _offset_dist_ if _offset_dist_ is not None \
        else 0.5 / conversion_to_meters()
    workers = 1 # _cpu_count_ if _cpu_count_ is not None else recommended_processor_count()

    # create the gridded mesh from the geometry
    study_mesh = to_joined_gridded_mesh3d(_geometry, _finesse)
    points = [from_point3d(pt.move(vec * _offset_dist_)) for pt, vec in
            zip(study_mesh.face_centroids, study_mesh.face_normals)]

    # mesh the geometry and context
    shade_mesh = join_geometry_to_mesh(_geometry + [context_])

    # deconstruct the matrix and get the sky dome vectors
    mtx_temp_IR = de_objectify_output(_sky_mtx_in_IR)
    total_sky_rad = [dir_rad + dif_rad for dir_rad, dif_rad in zip(mtx_temp_IR[1], mtx_temp_IR[2])]
    ground_rad = [(sum(total_sky_rad) / len(total_sky_rad)) * mtx_temp_IR[0][1]] * len(total_sky_rad)
    all_rad = total_sky_rad + ground_rad 
    lb_vecs = view_sphere.tregenza_dome_vectors if len(total_sky_rad) == 145 \
        else view_sphere.reinhart_dome_vectors
    if mtx_temp_IR[0][0] != 0:  # there is a north input for sky; rotate vectors
        north_angle = math.radians(mtx_temp_IR[0][0])
        lb_vecs = tuple(vec.rotate_xy(north_angle) for vec in lb_vecs)
    lb_grnd_vecs = tuple(vec.reverse() for vec in lb_vecs)
    all_vecs = [from_vector3d(vec) for vec in lb_vecs + lb_grnd_vecs]

    # intersect the rays with the mesh
    normals = [from_vector3d(vec) for vec in study_mesh.face_normals]
    int_matrix_init, angles = intersect_mesh_rays(
        shade_mesh, points, all_vecs, normals, cpu_count=workers)

    # compute the results
    results = []
    int_matrix = []
    for int_vals, angs in zip(int_matrix_init, angles):
        pt_rel = [ival * math.cos(ang) for ival, ang in zip(int_vals, angs)]
        int_matrix.append(pt_rel)
        rad_result = sum(r * w for r, w in zip(pt_rel, all_rad))
        results.append(rad_result)

    # convert to irradiance if requested -> NOT DONE
    if irradiance_:
        factor = 1000 / _sky_mtx_in_IR.wea_duration if hasattr(_sky_mtx_in_IR, 'wea_duration') \
            else 1000 / (((mtx_temp_IR[0][3] - mtx_temp_IR[0][2]).total_seconds() / 3600) + 1)
        print("factor :", factor)
        results = [r * factor for r in results]

    # output the intersection matrix and compute total radiation
    int_mtx_to_RTIR = objectify_output('Geometry/Sky Intersection Matrix', int_matrix)
    unit_conv = conversion_to_meters() ** 2
    total = 0
    for rad, area in zip(results, study_mesh.face_areas):
        total += rad * area * unit_conv
    return int_mtx_to_RTIR

Paramétrage/Settings :

In [6]:
# Geometrical settings  ====> ACTUELLEMENT CONFIGURE POUR PIOLENC
######
GRID_SIZE = 50 # Taille du champ de panneau pour la simulation
ENTRAXE = 2.25 # Espacement entre lignes de panneaux (= inter-rang classique, distance entre 2 supports de panneaux)
HAUTEUR = 6 # [0 ; 100] ; mètres ; hauter de l'axe de rotation, ou du point du PVP le plus bas.
RAMPANT = 1.134 # Largeur du panneau
NB_RANGS = 10 # Nombre de panneaux par ligne
NB_PVP_RANGS = int(GRID_SIZE / ENTRAXE) # Nombre de lignes
LONGUEUR_PVP = GRID_SIZE / (NB_RANGS) # Longueur des panneaux dans une même ligne
# Semitransp panels :
TYPE_PANEL = False #False = classic panel, True = semitransparent panel
LARGEUR_BANDE = 0.18
LARGEUR_AVIDE = (RAMPANT-4*LARGEUR_BANDE)/3

column_spacing = ENTRAXE # Espacement entre lignes de panneaux (= inter-rang)
row_spacing = GRID_SIZE / NB_RANGS # Espacement entre panneaux d'une même ligne (même support)

ANGLE_ORIENTATION = 10  # Rotation globale autour de l'axe Z (Orientation parcelle)

# Settings, year and hours
######
HEURE_DEB = 10
HEURE_FIN = 14
start_date = datetime(year=2023, month=1, day=1, hour=HEURE_DEB)  # Début de l'année (éviter les années bissextiles)
end_date = datetime(year=2024, month=1, day=1, hour=HEURE_DEB)  # Fin de l'année (début de l'année suivante)
NB_HEURES_ANNEES = (HEURE_FIN-HEURE_DEB)*(end_date-start_date).days

# Settings angles
#####
start_angle = -60
end_angle = 60
step_angle = 60
angles = np.arange(start_angle, end_angle + 1, step_angle)
angles = [-60]
print(list(angles))

# Settings radiance
#####
SKYMAT_FOLDER = "cielMatrices/"
HAUTEUR_MESURE = 1 # Hauteur de mesure dans IncidentRadiation
FINESSE = 1 # Finesse de la fonction IncidentRadiation (1 = peu précis, 0 = très précis)
_epw_file_path = "FRA_PR_Orange.Caritat.AB.075790_TMYx.epw" #Filepath to meteo file (EPW format)


[-60]


In [7]:
# #Generate all Rhino .3dm files corresponding to all the different tilt panels can have
# #Optionnal 

# panneaux_TEMOIN = rg.Brep()
# panneaux_TEMOIN.Append(create_surface(40, 40, 0.1, 0.1).ToBrep())
# sol_temoin = create_sensor_grid_2(ANGLE_ORIENTATION,40,40,culture=True)
# measures_grid = create_sensor_grid_2(ANGLE_ORIENTATION, None, None, culture=True, 
#                                     entraxe=ENTRAXE, rampant=RAMPANT, nb_lignes=NB_PVP_RANGS, grid_size=GRID_SIZE)
# for i in angles :
#     #VISU
#     file_path = "panneaux_PIOLENC/semitransp" + str(i) + ".3dm" 
#     p = create_panel_grid(GRID_SIZE, NB_RANGS, NB_PVP_RANGS, RAMPANT, LONGUEUR_PVP, HAUTEUR, column_spacing, row_spacing, ANGLE_ORIENTATION, i,
#                                         TYPE_PANEL, LARGEUR_BANDE, LARGEUR_AVIDE) #True/False : semitransparent panel or classic one
#     export_brep_to_file(p, file_path)
#     export_brep_to_file(measures_grid[0], file_path, False)
#     export_brep_to_file(panneaux_TEMOIN, file_path, False)
#     export_brep_to_file(sol_temoin[0], file_path, False)

Import EPW

In [8]:
# Here, the meteo file (epw format) is imported and necessary data for SkyMatrix construction is extracted
epw_data = epw.EPW(_epw_file_path)

location_ = epw_data.location #######
direct_normal_rad_ = epw_data.direct_normal_radiation #########
diffuse_horizontal_rad_ = epw_data.diffuse_horizontal_radiation #########

Boucle 1 (= main)

In [9]:
# # Creating the tab that will store every irradiation value, for every angle, for every hour of the year
# tab_hoys_angles = np.zeros((NB_HEURES_ANNEES, len(angles)))

# # The measurement grid does not change depending on the inclination of the panels, it can only be created once. True = arboriculture
# measures_grid = create_sensor_grid_2(ANGLE_ORIENTATION, None, None, culture=True, 
#                                     entraxe=ENTRAXE, rampant=RAMPANT, nb_lignes=NB_PVP_RANGS, grid_size=GRID_SIZE)

# # Crate first skymatrix (full year) to generate full year int_mtx
# s1 = create_sky_matrix(location_, direct_normal_rad_, diffuse_horizontal_rad_, SKYMAT_FOLDER)

# #Créer géom, int_mtx_TEMOIN et data pour zone témoin (une fois)
# panneaux_TEMOIN = rg.Brep()
# panneaux_TEMOIN.Append(create_surface(40, 40, 0.1, 0.1).ToBrep())
# sol_temoin = create_sensor_grid_2(ANGLE_ORIENTATION,40,40,culture=True)
# int_mtx_TEMOIN = np.array(de_objectify_output(incident_radiation(s1, sol_temoin, panneaux_TEMOIN, HAUTEUR_MESURE, FINESSE)), dtype=object)
# columns_TEMOIN = np.empty((0, len(int_mtx_TEMOIN)))

# #Boucle 1, Varying parameter : = tilt angle of PVPs
# for index, angle in enumerate(angles):

#     #Create panel geometry
#     final_rotated_brep = create_panel_grid(GRID_SIZE, NB_RANGS, NB_PVP_RANGS, RAMPANT, LONGUEUR_PVP, HAUTEUR, column_spacing, row_spacing, ANGLE_ORIENTATION, angle,
#                                            TYPE_PANEL, LARGEUR_BANDE, LARGEUR_AVIDE)
    
#     #Create int_mtx_angle that will be crossed with every hourly sky_mtx
#     int_mtx_angle = np.array(de_objectify_output(incident_radiation(s1, measures_grid, final_rotated_brep, HAUTEUR_MESURE, FINESSE)), dtype=object)

#     #Setting date
#     current_date = start_date

#     #Create table which will receive the X measurements for one hour for an angle /!\ PROCESS THIS DATA /!\
#     columns = np.empty((0, len(int_mtx_angle)))

#     #Create table that will receive the X values ​​for all hours of the year for an angle
#     irr_all_hoys_one_angle = np.zeros((NB_HEURES_ANNEES, 1))

#     while current_date < end_date:
#     ##############
#     ###BOUCLE 2### Varying parameter : date
#     ##############

#         if current_date.hour >= HEURE_FIN: #If current hour > HEURE_FIN, which is the last hour we want data 
#             #Skip to next day
#             current_date += timedelta(days=1)  #+1 day
#             current_date = current_date.replace(hour=HEURE_DEB)  #Set hour to starting hour (HEURE_DEB)
#             continue #Skip the rest of the loop
#         next_date = current_date + timedelta(hours=1) #next_date = current_date + 1h

#         ##Analysis Period##-----------------
#         anp = ap.AnalysisPeriod(
#             current_date.month, current_date.day, current_date.hour, # STARTING DATES
#             next_date.month, next_date.day, next_date.hour, # ENDING DATES
#             1) # TIMESTEP PER HOUR (SET TO 1)
#         hoys_sm = anp.hoys
#         ##Analysis Period##-----------------

#         ##Temporary SkyMatrix##-------------
#         sky_mtx_boucle = create_sky_matrix(location_,
#             direct_normal_rad_,
#             diffuse_horizontal_rad_,
#             SKYMAT_FOLDER,
#             _hoys_=hoys_sm)
#         ##Temporary SkyMatrix##-------------

#         ##LB RTIR##-------------------------
#         sky_mtx_temp_RTIR = np.array(de_objectify_output(sky_mtx_boucle), dtype = object)
        
#         total_sky_rad = sky_mtx_temp_RTIR[1] + sky_mtx_temp_RTIR[2]
#         average_sky_rad = np.mean(total_sky_rad)
#         ground_rad = average_sky_rad * sky_mtx_temp_RTIR[0][1] * np.ones_like(total_sky_rad)
#         all_rad = total_sky_rad + ground_rad
        
#         for pt_rel in int_mtx_angle:
#             results_RTIR = np.dot(int_mtx_angle, all_rad)
#         columns = np.vstack((columns, results_RTIR))
        
#         #-- Data control zone
#         if angle == list(angles)[0] :
#             for pt_rel2 in int_mtx_TEMOIN:
#                 results_RTIR_TEMOIN = np.dot(int_mtx_TEMOIN, all_rad)
#             columns_TEMOIN = np.vstack((columns_TEMOIN, results_RTIR_TEMOIN))
#         #-- Data control zone

#         ##LB RTIR##-------------------------
        
#         current_date += timedelta(hours=1)  # Increment by one hour
        
#     ##############
#     ###BOUCLE 2###
#     ##############

#     #Stocking the data in tab_hoys_angles
#     irr_all_hoys_one_angle = np.mean(columns, axis=1).reshape(-1, 1)
#     tab_hoys_angles[:, index] = irr_all_hoys_one_angle.squeeze()

# #Adding the control zone data
# irr_all_hoys_TEMOIN = np.mean(columns_TEMOIN, axis=1).reshape(-1, 1)
# tab_hoys_angles_temoin = np.column_stack((tab_hoys_angles, irr_all_hoys_TEMOIN))

In [10]:
# #VISU
# file_path = "output_apres_boucle.3dm"
# export_brep_to_file(final_rotated_brep, file_path)
# export_brep_to_file(measures_grid[0], file_path, False)
# export_brep_to_file(panneaux_TEMOIN, file_path, False)
# export_brep_to_file(sol_temoin[0], file_path, False)

# ----------------------------------------

In [11]:
from honeybee.model import Model
from honeybee.face import Face
from honeybee_radiance.sensorgrid import SensorGrid
from honeybee.typing import clean_and_id_string

from ladybug_geometry.geometry3d.face import Face3D
from ladybug_geometry.geometry3d.pointvector import Point3D
from ladybug_geometry.geometry3d.plane import Plane
from ladybug_geometry.geometry3d.face import Face3D
from ladybug_geometry.geometry3d.mesh import Mesh3D
from ladybug_rhino.togeometry import  to_mesh3d, to_face3d, to_vector3d
from ladybug_rhino.fromgeometry import from_point3d

from lbt_recipes.recipe import Recipe
from lbt_recipes.settings import RecipeSettings
from ladybug_rhino.togeometry import to_face3d
from ladybug.wea import Wea
from os import path


In [12]:
#FONCTIONS LADYBUGS DEBUGEES
def longest_list(values, index):
    """Get a value from a list while applying Grasshopper's longest-list logic.

    Args:
        values: An array of values from which a value will be pulled following
            longest list logic.
        index: Integer for the index of the item in the list to return. If this
            index is greater than the length of the values, the last item of the
            list will be returned.
    """
    try:
        return values[index]
    except IndexError:
        return values[-1]
    

def to_gridded_mesh3d_perso(brep, grid_size, offset_distance=0):
    """Create a gridded Ladybug Mesh3D from a Rhino Brep.

    This is useful since Rhino's grid meshing is often more beautiful than what
    ladybug_geometry can produce. However, the ladybug_geometry Face3D.get_mesh_grid
    method provides a workable alternative to this if it is needed.

    Args:
        brep: A Rhino Brep that will be converted into a gridded Ladybug Mesh3D.
        grid_size: A number for the grid size dimension with which to make the mesh.
        offset_distance: A number for the distance at which to offset the mesh from
            the underlying brep. The default is 0.
    """
    if not isinstance(brep, rg.Brep):  # it's likely an extrusion object
        brep = brep.ToBrep()  # extrusion objects must be cast to Brep in Rhino 8
    meshing_param = rg.MeshingParameters.Default
    meshing_param.MaximumEdgeLength = grid_size
    meshing_param.MinimumEdgeLength = grid_size
    meshing_param.GridAspectRatio = 1
    mesh_grids = rg.Mesh.CreateFromBrep(brep, meshing_param)
    if len(mesh_grids) == 1:  # only one mesh was generated
        mesh_grid = mesh_grids[0]
    else:  # join the meshes into one
        mesh_grid = rg.Mesh()
        for m_grid in mesh_grids:
            mesh_grid.Append(m_grid)
    if offset_distance != 0:
        temp_mesh = rg.Mesh()
        mesh_grid.Normals.UnitizeNormals()
        for pt, vec in zip(mesh_grid.Vertices, mesh_grid.Normals):
            # Startmodif
            vec3d = rg.Vector3d(vec) * offset_distance
            pt3d = rg.Point3d(pt)
            temp_mesh.Vertices.Add(pt3d + vec3d)
            # Endmodif
        for face in mesh_grid.Faces:
            temp_mesh.Faces.AddFace(face)
        mesh_grid = temp_mesh
    return to_mesh3d(mesh_grid)


def recipe_result(result):
    """Process a recipe result and handle the case that it's a list of list.

    Args:
        result: A recipe result to be processed.
    """
    if isinstance(result, (list, tuple)):
        return list_to_data_tree(result, s_type = str)
    return result


def list_to_data_tree(input, root_count=0, s_type = object):
    """Transform nested of lists or tuples to a Grasshopper DataTree.

    Args:
        input: A nested list of lists to be converted into a data tree.
        root_count: An integer for the starting path of the data tree.
        s_type: An optional data type (eg. float, int, str) that defines all of the
            data in the data tree. The default (object) works will all data types
            but the conversion to data trees can be more efficient if a more
            specific type is specified.
    """

    def proc(input, tree, track):
        for i, item in enumerate(input):
            if isinstance(item, (list, tuple, array.array)):  # ignore iterables like str
                track.append(i)
                proc(item, tree, track)
                track.pop()
            else:
                tree.Add(item, Path(*track))

    if input is not None:
        if not isinstance(s_type, type):
            raise TypeError("s_type must be a valid type")
        t = DataTree[s_type]()
        proc(input, t, [root_count])
        return t

In [13]:
#FONCTION ANNUAL IRRADIANCE
import os

# Fonction pour calculer l'irradiance annuelle
def annual_irradiance(_model, _wea, _timestep_ = 1, visible_ = False, north_ = 0, grid_filter_ = None, radiance_par_ = "-ab 2 -ad 5000 -lw 2e-05", run_settings_ = None):
    """
    Run an annual irradiance study for a Honeybee model to compute hourly solar
    irradiance for each sensor in a model's sensor grids.
    _
    The fundamental calculation of this recipe is the same as that of "HB Annual
    Daylight" in that an enhaced 2-phase method is used to accurately account for
    direct sun at each simulation step. However, this recipe computes broadband
    solar irradiance in W/m2 instead of visible illuminance in lux.
    _
    Consequently, the average irradiance and cumulative radiation values produced from
    this recipe are more accurate than those produced by the "HB Cumulative Radiation"
    recipe. Furthermore, because the hourly irriadiance values are accurate, this
    recipe can be used to evaluate `peak_irradiance` and determine the worst-case
    solar loads over clear sky Weas that represent cooling design days.

    -
        Args:
            _model: A Honeybee Model for which Annual Irradiance will be simulated.
                Note that this model must have grids assigned to it.
            _wea: A Wea object produced from the Wea components that are under the Light
                Sources tab. This can also be the path to a .wea or a .epw file.
            _timestep_: An integer for the timestep of the inpput _wea. This value is used
                to compute average irradiance and cumulative radiation. (Default: 1)
            visible_: Boolean to indicate the type of irradiance output, which can be solar (False)
                or visible (True). Note that the output values will still be
                irradiance (W/m2) when "visible" is selected but these irradiance
                values will be just for the visible portion of the electromagnetic
                spectrum. The visible irradiance values can be converted into
                illuminance by multiplying them by the Radiance luminous efficacy
                factor of 179. (Default: False).
            north_: A number between -360 and 360 for the counterclockwise difference
                between the North and the positive Y-axis in degrees. This can
                also be Vector for the direction to North. (Default: 0).
            grid_filter_: Text for a grid identifer or a pattern to filter the sensor grids of
                the model that are simulated. For instance, first_floor_* will simulate
                only the sensor grids that have an identifier that starts with
                first_floor_. By default, all grids in the model will be simulated.
            radiance_par_: Text for the radiance parameters to be used for ray
                tracing. (Default: -ab 2 -ad 5000 -lw 2e-05).
            run_settings_: Settings from the "HB Recipe Settings" component that specify
                how the recipe should be run. This can also be a text string of
                recipe settings.
            _run: Set to True to run the recipe and get results. This input can also be
                the integer "2" to run the recipe silently.

        Returns:
            results: Raw result files (.ill) that contain matrices of irradiance in W/m2
                for each time step of the wea.
    """
    # Créer la recette et définir les arguments d'entrée
    recipe = Recipe('annual-irradiance')
    recipe.input_value_by_name('model', _model)
    recipe.input_value_by_name('wea', _wea)
    recipe.input_value_by_name('timestep', _timestep_)
    recipe.input_value_by_name('output-type', visible_)
    recipe.input_value_by_name('north', north_)
    recipe.input_value_by_name('grid-filter', grid_filter_)
    recipe.input_value_by_name('radiance-parameters', radiance_par_)

    print('type run setting :',type(run_settings_))
    if run_settings_ is not None :
        project_folder = recipe.run(run_settings_, radiance_check=True, silent=False)

    print("type recipe :", type(recipe))


    recipe.default_project_folder = os.path.join('C:','Users','maceo.valente','Documents','Automatisation','TEST_MVA','TSAgriPV')
    
    print(recipe.default_project_folder)
    # load the results
    results = recipe_result(recipe.output_value_by_name('results', project_folder))

    return results


In [14]:
#FONCTIONS HONEYBEE GRID & MODEL
def combine_grid_model(_model, grids_, views_ = []):
    """
    Add radiance Sensor Grids and/or Views to a Honeybee Model.
    _
    This assignment is necessary for any Radiance study, though whether a grid or a
    view is required for a particular type of study is depenednet upon the recipe
    used.
    _
    Multiple copies of this component can be used in series and each will add the
    grids or views to any that already exist.

    -

        Args:
            _model: A Honeybee Model to which the input grids_ and views_ will be assigned.
            grids_: A list of Honeybee-Radiance SensorGrids, which will be assigned to
                the input _model.
            views_: A list of Honeybee-Radiance Views, which will be assigned to the
                input _model.

        Returns:
            model: The input Honeybee Model with the grids_ and views_ assigned to it.
"""
    assert isinstance(_model, Model), \
        'Expected Honeybee Model. Got {}.'.format(type(_model))
    model = _model.duplicate()  # duplicate to avoid editing the input
    if len(grids_) != 0: # add grids_ if non null
        model.properties.radiance.add_sensor_grids(grids_)
    if len(views_) != 0: # add views_ if non null
        model.properties.radiance.add_views(views_)

    return model

def convert_to_model(faces_, rooms_ = None, shades_ = None, apertures_ = None, doors_ = None, _name_ = "Grille"):
    """
    Create a Honeybee Model, which can be sent for simulation.
    -

        Args:
            rooms_: A list of honeybee Rooms to be added to the Model. Note that at
                least one Room is necessary to make a simulate-able energy model.
            faces_: A list of honeybee Faces to be added to the Model. Note that
                faces without a parent Room are not allowed for energy models.
            shades_: A list of honeybee Shades to be added to the Model.
            apertures_: A list of honeybee Apertures to be added to the Model. Note
                that apertures without a parent Face are not allowed for energy models.
            doors_: A list of honeybee Doors to be added to the Model. Note
                that doors without a parent Face are not allowed for energy models.
            _name_: Text to be used for the Model name and to be incorporated into a unique
                model identifier. If no name is provided, it will be "unnamed" and
                a unique model identifier will be auto-generated.

        Returns:
            report: Reports, errors, warnings, etc.
            model: A Honeybee Model object possessing all of the input geometry
                objects.
    """
    try:  # import the ladybug_rhino dependencies
        from ladybug_rhino.config import units_system, tolerance, angle_tolerance
        from honeybee.typing import clean_string, clean_and_id_string
    except ImportError as e:
        raise ImportError('\nFailed to import ladybug_rhino:\n\t{}'.format(e))
    
    # set a default name and get the Rhino Model units
    name = clean_string(_name_) if _name_ is not None else clean_and_id_string('unnamed')
    units = units_system()

    # create the model
    model = Model(name, rooms_, faces_, shades_, apertures_, doors_,
                units=units, tolerance=tolerance, angle_tolerance=angle_tolerance)

    return model

def brep_to_face(_geo,_name_ = [], _type_ = None, _bc_ = None):
    """
    Create Honeybee Face
    -

        Args:
            _geo: Rhino Brep or Mesh geometry.
            _name_: Text to set the name for the Face and to be incorporated into
                unique Face identifier. If the name is not provided, a random name
                will be assigned.
            _type_: Text for the face type. The face type will be used to set the
                material and construction for the surface if they are not assigned
                through the inputs below. The default is automatically set based
                on the normal direction of the Face (up being RoofCeiling, down
                being Floor and vertically-oriented being Wall).
                Choose from the following:
                    - Wall
                    - RoofCeiling
                    - Floor
                    - AirBoundary
            _bc_: Text for the boundary condition of the face. The boundary condition
                is also used to assign default materials and constructions as well as
                the nature of heat excahnge across the face in energy simulation.
                Default is Outdoors unless all vertices of the geometry lie below
                the below the XY plane, in which case it will be set to Ground.
                Choose from the following:
                    - Outdoors
                    - Ground
                    - Adiabatic
            ep_constr_: Optional text for the Face's energy construction to be looked
                up in the construction library. This can also be a custom OpaqueConstruction
                object. If no energy construction is input here, the face type and
                boundary condition will be used to assign a default.
            rad_mod_: Optional text for the Face's radiance modifier to be looked
                up in the modifier library. This can also be a custom modifier object.
                If no radiance modifier is input here, the face type and boundary
                condition will be used to assign a default.

        Returns:
            report: Reports, errors, warnings, etc.
            face: Honeybee faces. These can be used directly in radiance simulations
                or can be added to a Honeybee room for energy simulation.
    """
    faces = []  # list of faces that will be returned
    for j, geo in enumerate(_geo):
        if len(_name_) == 0:  # make a default Face name
            name = display_name = clean_and_id_string('Face')
        else:
            display_name = '{}_{}'.format(longest_list(_name_, j), j + 1) \
                if len(_name_) != len(_geo) else longest_list(_name_, j)
            name = clean_and_id_string(display_name)
        typ = None
        bc = None
        
        lb_faces = to_face3d(geo)
        for i, lb_face in enumerate(lb_faces):
            face_name = '{}_{}'.format(name, i) if len(lb_faces) > 1 else name
            hb_face = Face(face_name, lb_face, typ, bc)
            hb_face.display_name = display_name

            faces.append(hb_face)  # collect the final Faces
        
    return faces

def brep_to_pts_mesh(_geometry, _grid_size, _offset_dist_ = 1, quad_only_ = False):
    """
    Genrate a mesh with corresponding test points from a Rhino Brep (or Mesh).
    _
    The resulting mesh will be in a format that the "LB Spatial Heatmap" component
    will accept.
    -

        Args:
            _geometry: Brep or Mesh from which to generate the points and grid.
            _grid_size: Number for the size of the test grid.
            _offset_dist_: Number for the distance to move points from the surfaces
                of the input _geometry.  Typically, this should be a small positive
                number to ensure points are not blocked by the mesh. (Default: 0).
            quad_only_: Boolean to note whether meshing should be done using Rhino's
                defaults (False), which fills the entire _geometry to the edges
                with both quad and tringulated faces, or a mesh with only quad
                faces should be generated.
                _
                FOR ADVANCED USERS: This input can also be a vector object that will
                be used to set the orientation of the quad-only grid. Note that,
                if a vector is input here that is not aligned with the plane of
                the input _geometry, an error will be raised.

        Returns:
            points: Test points at the center of each mesh face.
            vectors: Vectors for the normal direction at each of the points.
            face_areas: Area of each mesh face.
            mesh: Analysis mesh that can be passed to the "LB Spatial Heatmap" component.
    """
    # check the input and generate the mesh.
    _offset_dist_ = _offset_dist_ or 0
    if quad_only_:  # use Ladybug's built-in meshing methods
        lb_faces = to_face3d(_geometry)
        try:
            x_axis = to_vector3d(quad_only_)
            lb_faces = [Face3D(f.boundary, Plane(f.normal, f[0], x_axis), f.holes)
                        for f in lb_faces]
        except AttributeError:
            pass  # no plane connected; juse use default orientation
        lb_meshes = []
        for geo in lb_faces:
            try:
                lb_meshes.append(geo.mesh_grid(_grid_size, offset=_offset_dist_))
            except AssertionError:  # tiny geometry not compatible with quad faces
                continue
        if len(lb_meshes) == 0:
            lb_mesh = None
        elif len(lb_meshes) == 1:
            lb_mesh = lb_meshes[0]
        elif len(lb_meshes) > 1:
            lb_mesh = Mesh3D.join_meshes(lb_meshes)
    else:  # use Rhino's default meshing
        lb_mesh = to_gridded_mesh3d_perso(_geometry, _grid_size, _offset_dist_)
        # except TypeError:  # assume it's a Rhino Mesh
        #     try:
        #         lb_mesh = to_mesh3d(_geometry)
        #     except TypeError:  # unidientified geometry type
        #         raise TypeError(
        #             '_geometry must be a Brep or a Mesh. Got {}.'.format(type(_geometry)))

    # generate the test points, vectors, and areas.
    if lb_mesh is not None:
        points = [from_point3d(pt) for pt in lb_mesh.face_centroids]

    return points

def convert_to_grid( _positions, _name_ = '', _directions_ = [], mesh_ = None, base_geo_ = None):
    """
    Create a Sensor Grid object that can be used in a grid-based recipe.
    -

        Args:
            _name_: A name for this sensor grid.
            _positions: A list or a datatree of points with one point for the position
                of each sensor. Each branch of the datatree will be considered as a
                separate sensor grid.
            _directions_: A list or a datatree of vectors with one vector for the
                direction of each sensor. The input here MUST therefor align with
                the input _positions. If no value is provided (0, 0, 1) will be
                assigned for all the sensors.
            mesh_: An optional mesh that aligns with the sensors. This is useful for
                generating visualizations of the sensor grid beyond the sensor
                positions. Note that the number of sensors in the grid must match
                the number of faces or the number vertices within the mesh.
            base_geo_: An optional Brep for the geometry used to make the grid. There are
                no restrictions on how this brep relates to the sensors and it is
                provided only to assist with the display of the grid when the number
                of sensors or the mesh is too large to be practically visualized.

        Returns:
            grid: An SensorGrid object that can be used in a grid-based recipe.
    """
    # set the default name and process the points to tuples
    pts = [(pt.X, pt.Y, pt.Z) for pt in _positions]

    # create the sensor grid object
    id  = _name_
    if len(_directions_) == 0:
        grid = SensorGrid.from_planar_positions(id, pts, (0, 0, 1))
        print("1 :",type(grid))
    else:
        vecs = [(vec.X, vec.Y, vec.Z) for vec in _directions_]
        grid = SensorGrid.from_position_and_direction(id, pts, vecs)
        print("2 :", type(grid))

    if mesh_ is not None:
        grid.mesh = to_mesh3d(mesh_)
    if base_geo_ is not None:
        grid.base_geometry = to_face3d(base_geo_)

    return grid

In [18]:
final_rotated_brep = create_panel_grid(GRID_SIZE, NB_RANGS, NB_PVP_RANGS, RAMPANT, LONGUEUR_PVP, HAUTEUR, column_spacing, row_spacing, ANGLE_ORIENTATION, 25,
                                       TYPE_PANEL, LARGEUR_BANDE, LARGEUR_AVIDE)
measures_grid = create_sensor_grid_2(ANGLE_ORIENTATION, None, None, culture=True, 
                                    entraxe=ENTRAXE, rampant=RAMPANT, nb_lignes=NB_PVP_RANGS, grid_size=GRID_SIZE)

a = [final_rotated_brep] #BREP
faces5 = brep_to_face(a) #BREP TO FACES
momo = convert_to_model(faces_= faces5) #FACES TO MODEL


pts_grid5 = brep_to_pts_mesh(measures_grid[0], _grid_size=0.5) #BREP TO POINTS

sensorgrid5 = convert_to_grid(pts_grid5,'ID1') #POINTS TO GRID
print("3 :",type(sensorgrid5))

fullmomo = combine_grid_model(momo, [sensorgrid5]) #MODEL COMBINED WITH GRID

print("4 :", type(fullmomo))


1 : <class 'honeybee_radiance.sensorgrid.SensorGrid'>
3 : <class 'honeybee_radiance.sensorgrid.SensorGrid'>
4 : <class 'honeybee.model.Model'>


In [19]:
timestep_ = 1
wea = Wea.from_epw_file(_epw_file_path, timestep_)

Running model

In [17]:
resultspath5 = os.path.join('Annual_Irr_Results')

settings = RecipeSettings(resultspath5)

print('annual_irradiance :')
results = annual_irradiance(_model=fullmomo, _wea= wea, run_settings_= settings)

annual_irradiance :
type run setting : <class 'lbt_recipes.settings.RecipeSettings'>
type recipe : <class 'lbt_recipes.recipe.Recipe'>
C:Users\maceo.valente\Documents\Automatisation\TEST_MVA\TSAgriPV
